In [1]:
import argparse
import numpy as np
import os.path
import scipy.linalg as sla
import sys
import datetime
import os
import psutil
import sparse
import operator

from scipy.sparse import coo_matrix

from dask.distributed import Client
from dask.bag import read_text
import dask.array as da

from sklearn.datasets import make_sparse_coded_signal

In [2]:
#Inputs
'''
These inputs were copied from the pyspark version.
I may not need all of these.

'''


input_data = ''
T = 20 #Rows
P = 20 #Cols

#Optional
R = int(0.07 * P) #Enforces sparcity 
M = 5 #Atoms of the learned dictionary
E = 0.01 #Epsilon
norm = True

print(f'R:\t{R}')
#Spark Options, look for similar Dask elements
partitions = 4*4  #Chunks?
execmem = '8g'    

#Outputs
file_D = 'D.txt'
file_z = 'z.txt'
prefix = False

R:	1


In [3]:
client = Client()

In [4]:
max_iterations = P*10
seed = np.random.randint(max_iterations +1, high = 4294967295)
S, D, SV = make_sparse_coded_signal(T,M,P,int(R),seed)
S = da.from_array(S)
S -= S.mean()
S /= sla.norm(S,axis = 0)


u_new = da.zeros(T)
v = da.zeros(P)

In [5]:
for m in range(M):
    #We need to broadcast the seed to create the initial random vector 
    #seed = np.random.randint(max_iterations +1, high = 4294967295)
    _SEED_ = client.scatter(seed, broadcast=True)
    np.random.seed(_SEED_.result())
    
    #Create a dense random vector
    u_old = da.random.random(T)
    u_old -= u_old.mean()
    u_old /= sla.norm(u_old,axis = 0)
    num_iterations = 0
    delta = 2*E
    
    # Start the inner loop: this learns a single atom.
    while num_iterations < max_iterations and delta > E:
        
        _U_ = client.scatter(u_old, broadcast=True) 
        v = da.dot(_U_.result(),S) #May get an error here because S may be a future instead of a dask array
        
        #Grab the indices and data of the top R values in v for the sparse vector
        indices = np.sort(v.argtopk(R,axis=0))
        data = v[indices].compute()  #Do I need to delete any of these intermediate variables? 
        
        #let's make the sparse vector.
        sv = sparse.COO(indices,data,shape=(P),sorted=True)
        sv = da.from_array(sv)
        
        # Broadcast the sparse vector.
        _V_ = client.scatter(sv,broadcast=True)

        # P1: Matrix-vector multiplication step. Computes u.
        u_new = da.dot(S,_V_.result()) 
        
        
        # Subtract off the mean and normalize.
        u_new -= u_new.mean()
        u_new /= sla.norm(u_new,axis = 0)
        u_new = u_new.compute()
        
        # Update for the next iteration.
        delta = sla.norm(u_old - u_new) #Should u_old be _U_?
        u_old = u_new
        num_iterations += 1
        
    # Save the newly-computed u and v to the output files;
    with open(file_D, "a+") as fD:
        np.savetxt(fD, u_new, fmt = "%.6f", newline = " ")
        fD.write("\n")
    with open(file_z, "a+") as fz:
        np.savetxt(fz, sv.compute().todense(), fmt = "%.6f", newline = " ")
        fz.write("\n")

    
    # P4: Deflation step. Update the primary data matrix S. 
    _U_ = client.scatter(u_new, broadcast=True)
    _V_ = client.scatter(sv, broadcast=True)

    
    #if args['debug']: print(m)
    print(m)
    
    S -= da.core.blockwise(operator.mul, 'ij', _U_.result(), 'i', _V_.result(), 'j', dtype='f8')
    
    S.persist()
    
    
    print(datetime.datetime.now())
    #if args['debug']: print(datetime.datetime.now())
    process = psutil.Process(os.getpid())
    print(process.memory_info().rss)

0
2020-01-28 23:44:03.918471
201437184
1
2020-01-28 23:44:04.588652
201437184
2
2020-01-28 23:44:05.260920
201633792
3
2020-01-28 23:44:05.959879
202264576
4
2020-01-28 23:44:06.787200
203100160


In [ ]:
read_text('*.txt').str.strip().str.split(' ').map(np.array).compute()
